<a href="https://colab.research.google.com/github/Axieyan/TextReID_XY/blob/main/Text_Based_Person_Search_with_Limited_Data_Xie10_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 下载原代码


In [1]:
# !git clone https://github.com/BrandonHanx/TextReID.git
# %cd TextReID
!git clone https://github.com/Axieyan/TextReID_XY.git TextReID
%cd TextReID

Cloning into 'TextReID'...
remote: Enumerating objects: 371, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 371 (delta 116), reused 108 (delta 108), pack-reused 227
Receiving objects: 100% (371/371), 190.33 KiB | 7.93 MiB/s, done.
Resolving deltas: 100% (216/216), done.
/content/TextReID


# 配置环境


In [2]:
!pip install tensorflow tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 30.2 MB/s 


In [3]:
!pip install -r requirements.txt
!pre-commit install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 37.5 MB/s 
     |████████████████████████████████| 496 kB 56.8 MB/s 
     |████████████████████████████████| 98 kB 8.0 MB/s 
     |████████████████████████████████| 15.7 MB 13.0 MB/s 
     |████████████████████████████████| 3.1 MB 52.0 MB/s 
     |████████████████████████████████| 191 kB 72.8 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3a6f2000 @  0x7f32c0c38615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |████████████████████████████████| 23.3 MB 88.9 MB/s 
     |█████████████████████

pre-commit installed at .git/hooks/pre-commit


In [4]:
!unzip '/content/drive/MyDrive/Subject1/datasets.zip' -d '/content/TextReID'

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6287_s8235.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6289_s8238.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6289_s8239.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6290_s8240.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6290_s8241.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6290_s8242.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6291_s8240.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6291_s8241.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6291_s8242.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6294_s8246.jpg  
  inflating: /content/TextReID/datasets/cuhkpedes/imgs/train_query/p6294_s8247.jpg  
  inflating: /content/TextReID/dataset

# 加载权重


In [5]:
!mkdir pretrained/clip/
!cd pretrained/clip
!wget https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt
!wget https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt
!cd -

mkdir: cannot create directory ‘pretrained/clip/’: No such file or directory
/bin/bash: line 0: cd: pretrained/clip: No such file or directory
--2022-10-07 09:32:02--  https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.59, 13.107.213.59, 2620:1ec:bdf::59, ...
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255827503 (244M) [application/octet-stream]
Saving to: ‘RN50.pt’

RN50.pt             100%[===================>] 243.98M   162MB/s    in 1.5s    

2022-10-07 09:32:03 (162 MB/s) - ‘RN50.pt’ saved [255827503/255827503]

--2022-10-07 09:32:03--  https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)...

In [6]:
!unzip '/content/drive/MyDrive/Subject1/pretrained.zip' -d '/content/TextReID'

Archive:  /content/drive/MyDrive/Subject1/pretrained.zip
   creating: /content/TextReID/pretrained/
   creating: /content/TextReID/pretrained/clip/
  inflating: /content/TextReID/pretrained/clip/RN101.pt  
  inflating: /content/TextReID/pretrained/clip/RN50.pt  


In [7]:
%cd /content/TextReID
!python train_net.py \
--config-file configs/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048.yaml \
--use-tensorboard

流式输出内容被截断，只能显示最后 5000 行内容。
2022-10-07 14:05:28,901 PersonSearch.trainer INFO: eta: 0:38:35  epoch [9][2907/3940]  loss: 29.7261 (39.7986)  instance_loss: 13.6370 (16.6080)  infonce_loss: 6.1432 (9.0475)  global_align_loss: 9.8333 (14.1430)  time: 0.4549 (0.4664)  data: 0.0287 (0.0416)  top1: 26.8681 (26.9859)  lr: 0.000100  max mem: 3140
2022-10-07 14:05:29,360 PersonSearch.trainer INFO: eta: 0:38:34  epoch [9][2908/3940]  loss: 29.6629 (39.7982)  instance_loss: 13.7465 (16.6080)  infonce_loss: 6.1061 (9.0474)  global_align_loss: 9.4789 (14.1429)  time: 0.4551 (0.4664)  data: 0.0290 (0.0416)  top1: 26.8681 (26.9859)  lr: 0.000100  max mem: 3140
2022-10-07 14:05:29,813 PersonSearch.trainer INFO: eta: 0:38:34  epoch [9][2909/3940]  loss: 29.6629 (39.7980)  instance_loss: 13.7465 (16.6079)  infonce_loss: 6.1432 (9.0473)  global_align_loss: 9.4789 (14.1428)  time: 0.4549 (0.4664)  data: 0.0287 (0.0416)  top1: 26.8681 (26.9859)  lr: 0.000100  max mem: 3140
2022-10-07 14:05:30,265 PersonSear

In [10]:
!cp /content/TextReID/output/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048/best.pth /content/drive/MyDrive/Data

# 查看模型数据


In [9]:
!python test_net.py \
--config-file configs/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048.yaml \
--checkpoint-file output/cuhkpedes/moco_gru_cliprn50_ls_bs128_2048/best.pth

loading annotations into memory...
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
2022-10-07 15:02:04,743 PersonSearch INFO: Using 1 GPUs
2022-10-07 15:02:04,743 PersonSearch INFO: AMP_VERBOSE: False
DATALOADER:
  EN_SAMPLER: True
  IMS_PER_ID: 4
  NUM_WORKERS: 4
DATASETS:
  TEST: ('cuhkpedes_test',)
  TRAIN: ('cuhkpedes_train',)
  USE_ONEHOT: True
DTYPE: float32
INPUT:
  HEIGHT: 384
  PADDING: 10
  PIXEL_MEAN: [0.48145466, 0.4578275, 0.40821073]
  PIXEL_STD: [0.26862954, 0.26130258, 0.27577711]
  USE_AUG: True
  WIDTH: 128
MODEL:
  DEVICE: cuda
  EMBEDDING:
    DROPOUT_PROB: 0.0
    